In [1]:
from pandas import read_csv
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import data
import lightgbm as lgb
from sklearn.metrics import log_loss # positive value

In [2]:
dataset = data.Dataset()
X_train = dataset.getTrain().values
X_test = dataset.getTest().values

y_train_data = read_csv('../y_train.csv')
y_train = y_train_data.values[:,1]
for i in range(len(y_train)):
    if y_train[i] == -1:
        y_train[i] = 0

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=441)

In [4]:
xgb_model = xgb.XGBClassifier(
                objective="multi:softmax", 
                # random_state=441, 
                # max_depth=8, 
                # max_leaves=66, 
                # learning_rate=0.1, 
                # subsample=0.8, 
                # min_child_weight = 8,
                # colsample_bytree = 0.6,
                # gamma = 0.1,
                # n_estimators = 200
            ).fit(X_train, y_train) 

In [5]:
# Using default num_leaves and learning_rate:
    # -0.846532 using {'feature_fraction': 0.26, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 175, 'num_leaves': 31, 'subsample': 0.5}
lgbm_model = lgb.LGBMClassifier(objective='multiclass', 
                                # num_class=5, boosting_type='gbdt', 
                                #     feature_fraction=0.26,
                                #     num_leaves=27, # Close to default
                                #     learning_rate=0.084, # Close to default
                                #     n_estimators=175,
                                #     max_depth=6,
                                #     subsample=0.5,
                                #     verbose=-1, force_col_wise=True
                                
                                # feature_fraction=1.0, learning_rate=0.1, max_depth= -1 (i.e. infty), n_estimators=100, num_leaves=31, subsample=1.0
                         ).fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.162852 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8618
[LightGBM] [Info] Number of data points in the train set: 38400, number of used features: 908
[LightGBM] [Info] Start training from score -4.286716
[LightGBM] [Info] Start training from score -1.562385
[LightGBM] [Info] Start training from score -1.218487
[LightGBM] [Info] Start training from score -1.267586
[LightGBM] [Info] Start training from score -1.612307


In [6]:
# Generate predicted probabilities from both models
xgb_probs = xgb_model.predict_proba(X_val)
lgb_probs = lgbm_model.predict_proba(X_val)

In [7]:
stacked = np.concatenate((xgb_probs, lgb_probs), axis=1)

In [8]:
#Test logistic meta-model
lmodel = LogisticRegressionCV(random_state=441, cv=5, max_iter=1000, scoring="neg_log_loss")
lmodel.fit(stacked,y_val)

LogisticRegressionCV(cv=5, max_iter=1000, random_state=441,
                     scoring='neg_log_loss')

In [9]:
print(lmodel.score(stacked, y_val))

-0.8647287263461764


In [15]:
#Test xgb meta-model
stack_model = xgb.XGBClassifier(
                objective="multi:softmax", 
                random_state=441, 
                # max_depth=8, 
                # max_leaves=66, 
                # learning_rate=0.1, 
                # subsample=0.8, 
                # min_child_weight = 8,
                # colsample_bytree = 0.6,
                # gamma = 0.1
)
#0.2 train/val
#Best: -0.849004 using {'colsample_bytree': 0.7, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 100, 'subsample': 0.8}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=441)
param_grid = {
    'max_depth': [2,3,4],
    'max_leaves': [2,3,4,5],
    'learning_rate': [0.07, 0.1, 0.13],
    'min_child_weight': [2,3,4],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.7, 0.8],
    'gamma': [0, 0.1, 0.2],
    'n_estimators': [100]
}
# param_grid = {
#     'max_depth': [2,3],
#     'max_leaves': [2,3],
#     'learning_rate': [0.1, 0.13, 0.15],
#     'min_child_weight': [4],
#     'subsample': [0.8],
#     'colsample_bytree': [0.7, 0.8],
#     'gamma': [0, 0.1, 0.2],
#     'n_estimators': [95, 100, 105]
# }

# param_grid = {
#     'colsample_bytree': [0.7], 
#     'gamma': [0], 
#     'learning_rate': [0.13], 
#     'max_depth': [2], 
#     'max_leaves': [2], 
#     'min_child_weight': [4], 
#     'subsample': [0.8],
#     'n_estimators': [100]
# }

grid_search = GridSearchCV(stack_model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(stacked, y_val)

print("\nBest: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# print results
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
 print("%f with: %r" % (mean, param))




Best: -0.849004 using {'colsample_bytree': 0.7, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 100, 'subsample': 0.8}
-0.856450 with: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.07, 'max_depth': 2, 'max_leaves': 2, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 0.8}
-0.857282 with: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.07, 'max_depth': 2, 'max_leaves': 2, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 0.9}
-0.858499 with: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.07, 'max_depth': 2, 'max_leaves': 2, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 1}
-0.856450 with: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.07, 'max_depth': 2, 'max_leaves': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.8}
-0.857282 with: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.07, 'max_depth': 2, 'max_leaves': 2, 'min_child_weight': 3, '

In [11]:
stacked_model = xgb.XGBClassifier(
                objective="multi:softmax", 
                random_state=441, 
                max_depth=2, 
                max_leaves=3, 
                learning_rate=0.1, 
                subsample=0.8, 
                min_child_weight = 4,
                colsample_bytree = 0.8,
                gamma = 0.1,
                n_estimators = 105
).fit(stacked, y_val)

In [12]:
xgb_test = xgb_model.predict_proba(X_test)
lgb_test = lgbm_model.predict_proba(X_test)
stacked_test = np.concatenate((xgb_test, lgb_test), axis=1)

In [13]:
prob = stacked_model.predict_proba(stacked_test)

In [16]:
test = np.c_[ np.ones(11438), prob ]              # add a column

for i in range(len(test)):
    test[i][0] = i

np.savetxt('stack_default.csv', test, delimiter=',', header="id,no answer,very important,quite important,not important,not at all important", comments="", fmt='%1.4f') 